In [45]:
import os, sys
os.environ['HADOOP_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['YARN_CONF_DIR'] = '/etc/hadoop/conf'
os.environ['PYSPARK_PYTHON'] =  'python3.9'
os.environ['PYSPARK_DRIVER_PYTHON'] = 'python3.9'
os.environ['HADOOP_USER_NAME']='senigov'

from pyspark.sql import SparkSession
from pyspark import SparkContext, SparkConf

In [46]:
conf = SparkConf().setAppName('ExpressInGroupSort').setMaster('yarn')
spark = SparkSession.builder.config(conf=conf).getOrCreate()

sc = spark.sparkContext
print("app_id".ljust(40), sc.applicationId)

app_id                                   application_1727681258360_0001


24/10/03 19:43:38 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [47]:
countries = [('fra', 'France', 68), ('bel', 'Belgium', 11), 
             ('usa', 'United States', 334), ('swe', 'Sweden', 10),
             ('ita', 'Italy', 59), ('chi', 'China', 1411),
             ('ind', 'India', 1425), ('bra', 'Brasil', 220),
             ('mex', 'Mexico', 126), ('rom', 'Romania', 19),
             ('jpn', 'Japan', 123)]

schema=["country", "country_name", "population"]

df_countries = spark.createDataFrame(data=countries, schema=schema)

In [48]:
from pyspark.sql.functions import avg, col, count, expr, when

df_countries.groupBy(
    [expr("case when population > 1000 then 'Giant' \
                when population > 200 then 'Very large' \
                when population > 50 then 'Large' \
                                     else 'Small' end").alias("population_flag"),
     when(col("country").isin(["chi", "ind", "jpn"]), 'Asia')\
                                       .otherwise("Not Asia").alias("asia_flag")]
  ).agg(
        avg("population").alias("average_population"),
        count("country").alias("number_countries") 
  ).orderBy( expr("case when population_flag = 'Giant' then 1 \
                        when population_flag = 'Very large' then 2 \
                        when population_flag = 'Large' then 3 \
                        else 4 end"), "asia_flag").show()


+---------------+---------+------------------+----------------+
|population_flag|asia_flag|average_population|number_countries|
+---------------+---------+------------------+----------------+
|          Giant|     Asia|            1418.0|               2|
|     Very large| Not Asia|             277.0|               2|
|          Large|     Asia|             123.0|               1|
|          Large| Not Asia| 84.33333333333333|               3|
|          Small| Not Asia|13.333333333333334|               3|
+---------------+---------+------------------+----------------+



In [ ]:
spark.stop()